# Forecasting Currency Price & Volume with DeepAR
## Part 2 - Train and Evaluate the Model

This lab will walk you through the main steps of tuning the hyperparameters required to train a Deep AR model. You will then then do a full training run of the Deep AR model using the discovered parameters. The goal is to create a single model that predicts multiple quantities 30 minutes into the future based on the previous 30 minutes of data. The values to be predicted include volume, high price, low price, open price and close price for the Australia dollar, Euro and Yen versus the US dollar.

To begin, modify the cell below so your generated s3 data will not interfere with other people working on the lab. Use the same user name and lab date from the previous lab.

In [ ]:
USER = <YOUR USER NAME IN QUOTES> # example 'whitefish'
BUCKET = <LAB BUCKET NAME in QUOTES>  # example 'grr.amazon.com-lab'

# override these if did not complete previous labs data prep
DATA_BUCKET = 'grr.amazon.com-lab' 
DATA_USER = 'example'
EPOCHS = 1

BEST_TRAINING_JOB = 'example-da-20190519030249'

### Imports & Configuration
First you need to import all the required python and SageMaker libraries and create a SageMaker session object. SageMaker uses the session to hold the local configuration state.

In [ ]:
import sys, boto3, time
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter

# get sagemaker session, image uri for the deep ar model and the execution role
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()

### Create Estimator
Now you can create an estimator. The estimator is the main entry point object upon which hyperparameter tuning, training and batch transform jobs can be created and run.

To configure the estimator, you need several things:

1. An IAM execution role that grants the estimator permissions to exeute Sage Maker functions.
2. The training instance type. Select a P2 instance.
3. The number of training instances.
4. The image name of the SageMaker model type. This could also be a docker image that holds your own model.
5. The job name for identification.
6. The s3 output path to store the model.
7. The Sage Maker session previously created.

In [ ]:
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')
role = get_execution_role()

job_name = USER + '-da-' + time.strftime('%Y%m%d%H%M%S', time.gmtime())
print('using job name '+job_name)

output_path = 's3://{}/labs/deepar/output/{}/models'.format(BUCKET, USER)
print ('creating deep ar models at '+output_path)

estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    base_job_name=job_name,
    output_path=output_path
)

#### Set Data Channels
You also need to set the data channels for training and testing. Data in test, train and holdout locations was generated in the previous tutorial. The buckets contain data in the format expected by deep ar. Since the previous tutorial didn’t complete due to time constraints, you will use a larger set previously generated of seven years of data for all three currencies.

In [ ]:
data_channels = {
    'train': 's3://{}/labs/deepar/output/{}/features/train'.format(DATA_BUCKET, DATA_USER),
    'test': 's3://{}/labs/deepar/output/{}/features/test'.format(DATA_BUCKET, DATA_USER),
}
print(data_channels)

Start Tuning Job
For a hyperparameter tuning job, you can choose which parameters are fixed, and which ones you wish the tuner to vary. The documentation for Deep AR recommends the quantities below, so you will use those ranges.

In [ ]:
# Set static values
estimator.set_hyperparameters(    
    time_freq = '1min',
    dropout_rate = 0.3,
    epochs = 600,
    prediction_length = '30',
    context_length = '30',
    likelihood = 'gaussian',
    early_stopping_patience = '10',
    mini_batch_size = 950,
    test_quantiles = [0.3, 0.4, 0.5, 0.6, 0.7]
)

# Configure hyperparameter ranges
hyp_tuner = HyperparameterTuner(
        estimator=estimator,  
        objective_metric_name='test:mean_wQuantileLoss',
        objective_type='Minimize',
        hyperparameter_ranges={
            'learning_rate': ContinuousParameter(0.0005, 0.01),
            'embedding_dimension': IntegerParameter(1, 10),
            'num_cells' : IntegerParameter(100,200),
            'num_layers': IntegerParameter(1,4)
        },
        max_jobs=50,
        max_parallel_jobs=1,
        early_stopping_type= 'Auto'
    )

# Run the tuning job
hyp_tuner.fit(data_channels, job_name=job_name)

#### Monitor Tuning
You can easily monitor the tuning job through API calls, or through the AWS console. The function below prints out basic details about the ongoing tuning job.

In [ ]:
from helper import monitor_tuner
monitor_tuner(job_name)

> **Note**
To monitor the executing job, you can also open the AWS console and go to `SageMaker -> Hyperparameter Tunning`

#### Stop Tuning Job
Since the tuning job is a long running task (over 20 hours), you will now stop it. The next steps in the lab will use the values from a job that was run previously.

In [ ]:
hyp_tuner.stop_tuning_job()

### Train the Model
For re-training a model on new data, you can skip hyperparameter tuning, and use the same hyperparameters values. When the data changes significantly, you can always re-run a hyperparameter tuning job, with a tighter range of values around what worked best last time. The steps to run a training job are very similar to the hyperparameter job, the difference being you specify static parameters as opposed to ranges. Execute the code below to start a training job.

In [ ]:
estimator.set_hyperparameters(    
        time_freq = '1min',
        context_length = '30',
        prediction_length = '30',
        num_cells = '132',
        num_layers = '1',
        likelihood = 'gaussian',
        epochs = EPOCHS,
        mini_batch_size = '600',
        learning_rate = '0.008',
        dropout_rate = '0.3',
        early_stopping_patience = '20',
        embedding_dimension = '1',
        test_quantiles = [0.3, 0.4, 0.5, 0.6, 0.7]
    )

estimator.fit(data_channels, wait=False, job_name=job_name)
print('training '+job_name)

#### Stop Training Job
Since the training job is a long running task (over 1 hour), you will now stop it. The training call is synchronous, so you will need to hit the stop button on the notebook before preceding. This does not stop the server-side training job. So be sure to execute the stop call below. The next steps in the lab will use a model that was trained previously, so we don’t need to wait for training.

In [ ]:
boto3.client('sagemaker', region_name=region).stop_training_job(TrainingJobName=job_name)

### Create an Endpoint
Now that you have a trained model, you can create an endpoint that can be used for scoring in real time. To create the endpoint, you need the job name of the best training run. We will use a previously trained model for this exercise.

> **Note** If only batch scoring is needed, you should not create an endpoint. An endpoint is always running and will be billed according to the on demand instance type it is deployed on. Batch jobs are also billed by instance type, but only for the duration of the batch job.

In [ ]:
# attach to best model from previous HP tuning job
estimator = Estimator.attach(BEST_TRAINING_JOB)

# example endpoint already deployed, skip this
#predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

### Evaluate the Model

The best way to evaluate the model perfromance is to graph actuals versus predicted over a wide range of varying time series. 

In [ ]:
import helper
from helper import predict_from_file, graph_predictions

s3 = boto3.client('s3')

# find a non-empty holdout data file
path = 'labs/deepar/output/{}/features/holdout/'.format(DATA_USER)
response = s3.list_objects_v2(Bucket=DATA_BUCKET, MaxKeys=100, Prefix=path) 
for content in response['Contents']:
    key = content['Key'].split('/')[-1]
    if key.split('.')[-1] == 'json' and int(content['Size']) > 100:
        break

holdout_file = key

# make some predictions
requests, actuals, predicteds = predict_from_file(sagemaker_session, BEST_TRAINING_JOB, DATA_BUCKET, path, holdout_file, 30, 30, quantiles=["0.3", "0.4", "0.5", "0.6", "0.7"])
graph_predictions(requests, actuals, predicteds)

#### Delete Endpoint
Since you will be using batch scoring, you will now delete the endpoint.

In [ ]:
boto3.client('sagemaker', region_name=region_name).delete_endpoint(EndpointName=endpoint_name)

### Batch Score
In may be more effective and economical to use batch scoring on a large set of files. The code below demonstrates how to use the estimator to create a transform job.

In [ ]:
holdout_dir = 's3://{}/labs/deepar/output/{}/features/holdout'.format(DATA_BUCKET, DATA_USER)
predictions_dir = 's3://{}/labs/deepar/output/{}/predictions'.format(BUCKET, USER)
print ('transforming {} to {}'.format(holdout_dir, predictions_dir))

# create transformer
transformer = estimator.transformer(
        instance_count=1,
        instance_type='ml.c4.xlarge',
        output_path=predictions_dir,
        env = {'output_types': 'quantiles'}
)

# transform holdout set to predictions
transformer.transform(holdout_dir, split_type='Line')

#### Stop Transform Job
Since the transform job is a long running task (over 1 hour), you will now stop it.

In [ ]:
boto3.client('sagemaker', region_name=region_name).stop_transform_job(TransformJobName=transformer._current_job_name)